In [3]:
import altair as alt
import ibis_vega_transform
import ibis.omniscidb


conn = ibis.omniscidb.connect(
    host='bewdy.mapd.com', user='mapd', password='HyperInteractive',
    port=6274, database='mapd', protocol= 'binary'
)

In [2]:
# conn.create_table('github', schema=ibis.schema([
#     ('type', 'string'),
#     ('public_', 'boolean'),
#     ('repo_id', 'int64'),
#     ('repo_name', 'string'),
#     ('repo_url', 'string'),
#     ('actor_id', 'int64'),
#     ('actor_login', 'int64'),
#     ('actor_gravatar_id', 'string'),
#     ('actor_avatar_url', 'string'),
#     ('actor_url', 'string'),
#     ('org_id', 'int64'),
#     ('org_login', 'string'),
#     ('org_gravatar_id', 'string'),
#     ('org_avatar_url', 'string'),
#     ('org_url', 'string'),
#     ('created_at', 'timestamp'),
#     ('id', 'string'),
# ]))

In [4]:
t = conn.table('github')

In [9]:
!pip install graphviz

In [12]:
ibis.options.graphviz_repr

True

In [11]:
t.head()._repr_png_()

In [13]:
import ibis.expr.visualize

In [ ]:
!pip install -U ibis

In [15]:
t.head().visualize()

TypeError: pipe() missing 1 required positional argument: 'f'

In [5]:
t.head().execute()

,type,public_,repo_id,repo_name,repo_url,actor_id,actor_login,actor_gravatar_id,actor_avatar_url,actor_url,org_id,org_login,org_gravatar_id,org_avatar_url,org_url,created_at,id
0,PushEvent,True,28684182,kasperbasse/kbwebs-webshop,https://api.github.com/repos/kasperbasse/kbweb...,1561022,None,None,https://avatars.githubusercontent.com/u/1561022?,https://api.github.com/users/kasperbasse,NaN,None,None,None,None,2015-01-01 10:10:29,2489549404
1,PushEvent,True,27468966,xsatter/timeslicerfx,https://api.github.com/repos/xsatter/timeslicerfx,6720685,None,None,https://avatars.githubusercontent.com/u/6720685?,https://api.github.com/users/xsatter,NaN,None,None,None,None,2015-01-01 10:10:29,2489549406
2,CreateEvent,True,28685069,wpeasyplugins/wpeasyplugins,https://api.github.com/repos/wpeasyplugins/wpe...,2277443,None,None,https://avatars.githubusercontent.com/u/2277443?,https://api.github.com/users/kjellberg,10363551.0,wpeasyplugins,None,https://avatars.githubusercontent.com/u/10363551?,https://api.github.com/orgs/wpeasyplugins,2015-01-01 11:01:26,2489564564
3,PushEvent,True,23782919,hduffddybz/MyDocument,https://api.github.com/repos/hduffddybz/MyDocu...,8110714,None,None,https://avatars.githubusercontent.com/u/8110714?,https://api.github.com/users/hduffddybz,NaN,None,None,None,None,2015-01-01 10:10:29,2489549408
4,CreateEvent,True,28684757,pgilad/gulp-sort,https://api.github.com/repos/pgilad/gulp-sort,4533329,None,None,https://avatars.githubusercontent.com/u/4533329?,https://api.github.com/users/pgilad,NaN,None,None,None,None,2015-01-01 10:35:54,2489557091


In [7]:
repo_dropdown = alt.binding_select(options=[
    'jupyterlab/jupyterlab',
    'vega/vega',
    'vega/vega-lite',
    'altair-viz/altair',
    'cloudera/ibis',
    'mapd/mapd-core'
])
repo_selection = alt.selection_single(fields=['repo_name'], bind=repo_dropdown)

time_selection = alt.selection_interval(
    fields=['created_at'],
    encodings=['x'],
)


time = alt.Chart(
    t,
).mark_line().encode(
    alt.X(
        'yearmonth(created_at):T',
    ),
    alt.Y(
        'count():Q',
        scale=alt.Scale(zero=False)
    ),
).add_selection(
    repo_selection
).transform_filter(
    repo_selection
).properties(
    height=600,
    width=600
).add_selection(
    time_selection
)

type_ = alt.Chart(
    t,
).mark_bar().encode(
    alt.X(
        'count()',
    ),
    alt.Y('type:O', sort=alt.Sort(encoding='x', order='descending')),
).add_selection(
    repo_selection
).transform_filter(
    repo_selection
).properties(
    height=600,
    width=600
).transform_filter(
    time_selection
)

(time & type_).properties(
    padding={
        "left": 150,
        "right": 0,
        "top": 10,
        "bottom": 50
    }
)

alt.VConcatChart(...)

In [ ]:

# alt.Chart(
#     t[t.repo_name].head().execute(),
# ).mark_line().encode(
#     y=alt.Y('repo_name', sort=alt.Sort(encoding='x', order='descending')),
#     x='count()',
# ).properties(
#     height=600,
#     width=600
# ).transform_window(
#     rank='rank(IMDB_Rating)',
#     sort=[alt.SortField('IMDB_Rating', order='descending')]
# ).transform_filter(
#     (alt.datum.rank < 10)
# )

In [26]:
repo_dropdown = alt.binding_select(options=[
    'jupyterlab/jupyterlab',
    'vega/vega',
    'vega/vega-lite',
    'altair-viz/altair',
    'cloudera/ibis',
    'mapd/mapd-core'
])
repo_selection = alt.selection_single(fields=['repo_name'], bind=repo_dropdown)

alt.Chart(
    t[t.created_at, t.repo_name, t.type],
).mark_bar().encode(
    alt.X(
        'count()',
    ),
    alt.Y('type:O', sort=alt.Sort(encoding='x', order='descending')),
    tooltip='type'
).add_selection(
    repo_selection
).transform_filter(
    repo_selection
).properties(
    height=600,
    width=600,
    padding={
        "left": 150,
        "right": 0,
        "top": 0,
        "bottom": 50
    }
)

alt.Chart(...)

In [24]:
# this doesn't work, input in this instead:
# omnisql --db mapd -u mapd -p HyperInteractive -s bewdy.mapd.com

# conn.sql("""
# COPY github
# FROM '/theHoard/export/internal-data/datasets/github-archive/processed/*/*.csv.gz'
# WITH (header='false');
# """)

In [32]:
# conn = ibis.omniscidb.connect(
#     host='localhost', password='HyperInteractive', user='admin',
#     port=6274,  protocol= 'binary'
# )

In [34]:
# t = conn.table('github_demo')

In [9]:
# t.info()

In [10]:
# t.head().execute()